In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()


2019-02-19 00:19:19,134 utils 400 [INFO]    [logger_func] start 


In [18]:
set_no = 4

#========================================================================
# Args
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
#========================================================================

#========================================================================
# Data Load
win_path = f'../features/4_winner/*.gz'
model_path_list = [f'../model/LB3670_70leaves_colsam0322/*.gz', '../model/E2_lift_set/*.gz', '../model/E3_PCA_set/*.gz', '../model/E4_mix_set/*.gz']
model_path = model_path_list[set_no-1]
# win_path_list = glob.glob(model_path) + glob.glob(win_path)
win_path_list = glob.glob(model_path)

base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target]]
# base_train = base[~base[target].isnull()].reset_index(drop=True)
# base_test = base[base[target].isnull()].reset_index(drop=True)

feature_list = utils.parallel_load_data(path_list=win_path_list)
df_feat = pd.concat(feature_list, axis=1)
df_feat = pd.concat([base, df_feat], axis=1)
#========================================================================
print(df_feat.shape)
df_feat.head()

(325540, 292)


,card_id,target,411_ker_new_weekend_mean@,400_ker_hist_price_mean@,410_ker_auth_category_2_4_0_mean@,412_rgt_lift_hist_regist_term_auth1_cat1_purchase_amount_sum@,490_fe1_all_auth1_installments_sum@,412_rgt_lift_hist_regist_term_auth0_cat0_installments_mean@,490_fe1_all_auth0_term@,406_pst_auth1_cat1_monthly_avg_purchase_amount_min@,...,412_rgt_lift_hist_regist_term_auth0_cat0_month_lag_mean@,412_rgt_lift_hist_regist_term_auth1_cat0_card_id_size@,490_fe1_all_auth1_monthly_avg_purchase_amount_sum@,403_pst_diff_new_auth1_installments_var@,410_ker_hist_category_1_mean@,452_ah0__auth0_hist_month_lag_var@,433_lgr_lift_hist_purchase_month_max_auth1_merchant_id_nunique@,400_ker_hist_price_min@,490_fe1_all_auth0_installments_sum@,412_rgt_lift_hist_regist_term_auth1_cat0_subsector_id_nunique@
0,C_ID_92a2005557,-0.820283,0.260870,-1.745371,0.000000,-130.463547,0.0,0.259160,164.0,-1.746893,...,0.876994,2.630905,-17.488558,0.000000,0.000000,2.769231,2.610739,-1.800000,4.0,1.781877
1,C_ID_3d0044924f,0.392913,0.000000,-0.574578,0.000000,1.380244,501.0,2.901153,299.0,-0.434138,...,0.808111,3.301946,-16.067957,-1.824213,0.181818,7.272727,5.526933,-0.742400,42.0,1.781877
2,C_ID_d639edf6cd,0.688056,1.000000,-1.745371,0.000000,-130.463547,0.0,0.000000,56.0,-1.746893,...,1.837969,0.358489,-2.319035,-1.000000,0.000000,2.000000,0.315584,-1.800000,0.0,0.569965
3,C_ID_186d6a6901,0.142495,0.428571,-0.604956,0.688312,0.801663,84.0,-2.268757,-1.0,-0.636944,...,-1.000000,1.000331,-8.248561,0.224539,-1.000000,-1.000000,1.377093,-0.740897,-27.0,0.948889
4,C_ID_cdbd2c0db2,-0.159749,0.333333,-0.590465,0.820312,-1.446962,144.0,0.945663,37.0,-0.323190,...,0.711854,3.171059,-17.260117,-0.865811,0.600000,0.300000,1.807435,-0.746156,38.0,1.385923


In [19]:
# df_feat.to_csv('../output/0219_go_elo_feature_Set2.csv', index=False)

# Set2
# utils.to_pkl_gzip(obj=df_feat, path=f'../output/0219_go_elo_feature_Set{set_no}_lift_main.csv')
# Set3
# utils.to_pkl_gzip(obj=df_feat, path=f'../output/0219_go_elo_feature_Set{set_no}_PCA_main.csv')
# Set4
utils.to_pkl_gzip(obj=df_feat, path=f'../output/0219_go_elo_feature_Set{set_no}_by_cleansing_dataset.csv')

In [9]:
base = utils.read_pkl_gzip('../input/base_no_out_clf.gz')[[key, target, 'no_out_flg', 'clf_pred']]
base.to_csv('../output/0219_go_elo_classifier_pred_NoOutlierFlg.csv', index=False)